<h1>Gather and Clean Data</h1>

<h4>Import Dependencies</h4>

In [1]:
import requests 
import json
from pprint import pprint
import os
import pandas as pd
import csv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

<h4>Read in GDP csv data</h4>

In [2]:
# Import GDP data
path_gdp = os.path.join("Resources", "GDP_1991_to_2016.csv")
gdp_df = pd.read_csv(path_gdp, encoding = "ISO-8859-1", engine='python')

# Renaming columns
gdp_df = gdp_df.rename(columns={'ï»¿Country Name': 'Country', 'Country Code': 'ISO3'})

# Getting rid of columns we don't need. We use only data between 2005 - 2015
gdp_df = gdp_df[['Country', 'ISO3', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', 
                    '2015']]
gdp_df.head()

,Country,ISO3,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,ABW,2.330726e+09,2.424581e+09,2.615084e+09,2.745251e+09,2.498883e+09,2.390503e+09,2.549721e+09,2.534637e+09,2.701676e+09,2.765363e+09,2.919553e+09
1,Afghanistan,AFG,6.209138e+09,6.971286e+09,9.747880e+09,1.010923e+10,1.243909e+10,1.585657e+10,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10
2,Angola,AGO,3.697092e+10,5.238101e+10,6.526645e+10,8.853861e+10,7.030716e+10,8.379950e+10,1.117900e+11,1.280530e+11,1.367100e+11,1.457120e+11,1.161940e+11
3,Albania,ALB,8.052076e+09,8.896074e+09,1.067732e+10,1.288135e+10,1.204422e+10,1.192693e+10,1.289077e+10,1.231983e+10,1.277622e+10,1.322814e+10,1.138685e+10
4,Andorra,AND,3.159905e+09,3.456442e+09,3.952601e+09,4.085631e+09,3.674410e+09,3.449967e+09,3.629204e+09,3.188809e+09,3.193704e+09,3.271808e+09,2.789870e+09


In [16]:
# Import Weather data
# Import temperature data
path_temp = os.path.join("Resources", "Temp_Data.csv")
temp_df = pd.read_csv(path_temp, encoding = "ISO-8859-1", engine='python')

# rename columns
temp_df = temp_df.rename(columns={'Temp_C': 'Temperature_Celsius', ' Year': 'Year', ' Statistics': 'Statistics', ' Country': 'Country', ' ISO3': 'ISO3'})
temp_df.columns

# # # Convert Temps from C to F
temp_df['Temp_Farenheit'] = ((temp_df['Temperature_Celsius']*(9/5)+32))
temp_df.head()

#  Get average temp per yr for eeach country
meanTemp_df = temp_df.groupby(['Year','Country', 'ISO3'])['Temp_Farenheit'].mean()
meanTemp_df

# # Import rainfall data
rain_df = pd.read_csv('Resources/Precip_Data.csv')
rain_df.columns
# rename columns
rain_df = rain_df.rename(columns={'Rainfall - (MM)': 'Rainfall_mm', ' Year': 'Year', ' Statistics': 'Statistics', ' Country': 'Country', ' ISO3': 'ISO3'})
rain_df.columns

#  Get average temp per yr for eeach country
meanRain_df = rain_df.groupby(['Year','Country', 'ISO3'])['Rainfall_mm'].mean()
meanRain_df

# # # # Combine temp and weather data into a single dataframe
weather_df = pd.merge(meanTemp_df, meanRain_df, how='left', on=['ISO3', 'Year', 'Country'])
weather_df = weather_df.sort_values(by=['ISO3', 'Year'])
weather_df = weather_df.reset_index()
weather_df = weather_df.dropna(how='any')

#  Crop data between 2005 - 2015
YrsList = [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
weather_df = weather_df[weather_df["Year"].isin(YrsList)]
CleanCountryList = [x.strip() for x in weather_df['Country'].tolist()]
CleanISO3List = [x.strip() for x in weather_df['ISO3'].tolist()]

weather_df['Country_clean'] = CleanCountryList
weather_df['ISO3_clean'] = CleanISO3List
weather_df = weather_df[['ISO3_clean', 'Country_clean', 'Year', 'Temp_Farenheit', 'Rainfall_mm']]
weather_df = weather_df.rename(columns={'Country_clean': 'Country', 'ISO3_clean': 'ISO3' })
weather_df

weather_df.to_csv('Resources/weather_df_2005-2015.csv')

<h4>Read in Country ISO3 Codes from API source and store it in a dictionary</h4>

In [4]:
import requests

code_dict = {}
my_request_thing = [x.strip() for x in weather_df['Country'].unique().tolist()]
for name in my_request_thing:
    url = f"https://restcountries.eu/rest/v2/name/{name}?fullText=true"
    response = requests.get(url)
    print(name, end='\r')
    try:
        code_dict[name] = response.json()[0]['alpha3Code']
    except KeyError:
        code_dict[name] = None
code_dict

{'Afghanistan': 'AFG',
 'Angola': 'AGO',
 'Albania': 'ALB',
 'Andorra': 'AND',
 'United Arab Emirates': 'ARE',
 'Argentina': 'ARG',
 'Armenia': 'ARM',
 'Antigua and Barbuda': 'ATG',
 'Australia': 'AUS',
 'Austria': 'AUT',
 'Azerbaijan': 'AZE',
 'Burundi': 'BDI',
 'Belgium': 'BEL',
 'Benin': 'BEN',
 'Burkina Faso': 'BFA',
 'Bangladesh': 'BGD',
 'Bulgaria': 'BGR',
 'Bahrain': 'BHR',
 'Bahamas': 'BHS',
 'Bosnia and Herzegovina': 'BIH',
 'Belarus': 'BLR',
 'Belize': 'BLZ',
 'Bolivia': None,
 'Brazil': 'BRA',
 'Barbados': 'BRB',
 'Brunei': None,
 'Bhutan': 'BTN',
 'Botswana': 'BWA',
 'Central African Republic': 'CAF',
 'Canada': 'CAN',
 'Switzerland': 'CHE',
 'Chile': 'CHL',
 'China': 'CHN',
 "Cote d'Ivoire": 'CIV',
 'Cameroon': 'CMR',
 'Congo (Democratic Republic of the)': 'COD',
 'Congo (Republic of the)': None,
 'Colombia': 'COL',
 'Comoros': 'COM',
 'Cape Verde': None,
 'Costa Rica': 'CRI',
 'Cuba': 'CUB',
 'Cyprus': 'CYP',
 'Czech Republic': 'CZE',
 'Germany': 'DEU',
 'Djibouti': 'DJI'

In [10]:
# Get GDP Data as a dict of lists for each country plotting
YearGDPList = []
YearGDPDict = {}
gdp_edit_df = gdp_df.sort_values(by=['Country_Code'])
for index, rows in gdp_edit_df.iterrows():
    # Create list for the current row 
    CountryGDP_list =[rows['2005_GDPData'], 
                     rows['2006_GDPData'], rows['2007_GDPData'], rows['2008_GDPData'], rows['2009_GDPData'], rows['2010_GDPData'], 
                     rows['2011_GDPData'], rows['2012_GDPData'], rows['2013_GDPData'], rows['2014_GDPData'], rows['2015_GDPData']]
    
    # append the list to the final Dict 
    YearGDPDict[rows.Country_Name] = CountryGDP_list
YearGDPDict    

# Make the lists I need
YrRows_step1 = weather_df[['ISO3', 'Country', 'Year']]
YrRows_step2 = YrRows_step1.drop_duplicates(subset = ['Country', 'ISO3', 'Year'])
CountryList = YrRows_step2['Country'].unique()
ISO3List = YrRows_step2['ISO3'].unique()
YrsList = YrRows_step2['Year'].unique()

# Get Temperature Data as a dict of lists for each country plotting
CountryTempList = []
CountryTempDict = {}
for country in CountryList:
    weather_df = weather_df.sort_values(by=['ISO3'])
    Country_dfv1 = weather_df.loc[(weather_df['Year']>=2005) & (weather_df['Year']<=2015)]
    Country_dfv1 = weather_df.loc[weather_df['Country']==country]
    Country_df = Country_dfv1.sort_values(by=['Year'])
    CountryTempList = Country_df['Temp_Farenheit'].values.tolist()
    CountryTempDict[country] = CountryTempList

CountryTempDict


# Get Rainfall Data as a dict of lists for each country plotting
CountryRainList = []
CountryRainDict = {}
for country in CountryList:
    weather_df = weather_df.sort_values(by=['ISO3'])
    Countryr_dfv1 = weather_df.loc[(weather_df['Year']>=1991) & (weather_df['Year']<=2017)]
    Countryr_dfv1 = weather_df.loc[weather_df['Country']==country]
    Countryr_df = Countryr_dfv1.sort_values(by=['Year'])
    CountryRainList = Country_df['Rainfall_mm'].values.tolist()
    CountryRainDict[country] = CountryRainList

CountryRainDict

KeyError: 'Country_Code'

In [ ]:
# # # Merge GDP and Weather data
# dict_with_lists_GDP = {"GDP": YearGDPDict, "Temp": CountryTempDict, "Rain": CountryRainDict}

# Yrly_GDP_df = pd.DataFrame(dict_with_lists_GDP)
# Yrly_GDP_df

# # TestList = [x.strip() for x in weather_df['ISO3'].unique().tolist()]
# # TestList

testA = weather_df.groupby(['Year', 'Country'])
testA.head()

In [38]:
# # Getting weather and GDP data to work together
# # Create a working copy
gdp2_df = gdp_df.copy()
ListofISO3 = weather_df['ISO3']

# Get GDP data to look like weather data
gdp2_df = gdp2_df.set_index('ISO3')
gdp2_df.columns = gdp2_df.columns.str.split(expand=True)
gdp2_df.columns.names = ['Year']
gdp2_df_output = gdp2_df.stack(0).reset_index()
gdp2_df_output.head(50)

# # Set index, remove unwanted stuff, rename columns and sort
# weather_df.set_index('Country', inplace=True)
weather_df = weather_df.sort_values(by=['Country', 'Year'])

# gdp2_df_output.set_index('Country', inplace=True)
gdp_v2_df = gdp2_df_output.sort_values(by=['ISO3', 'Year'])
gdp_v2_df = gdp_v2_df.loc[gdp2_df_output['Year']!= "Country"]
gdp_v2_df = gdp_v2_df.rename(columns={0: 'GDP'})
gdp_v2_df

# Convert to float
gdp_v2_df['GDP'] = pd.to_numeric(gdp_v2_df['GDP'])
gdp_v2_df['Year'] = pd.to_numeric(gdp_v2_df['Year'])
# gdp2_df_output['GDP'].astype(str).astype(int)
# gdp_v2_df.dtypes
# gdp2_df_output = gdp2_df_output.convert_dtypes()
# gdp2_df_output.dtypes

# # Clean GDP data to match Weather data
# # for index, row in gdp_v2_df.iterrows():
# #     if gdp_v2_df['Country'] in ListofCountries:

# # gdp_v2_df = gdp_v2_df[gdp_v2_df['ISO3'].isin(ListofISO3)]
# gdp_v2_df.columns

# # # # # Clean Weather data to match GDP data (# of Year available)
# # # for yr in PlotYrList:
# # #     weather_df = weather_df.loc[weather_df['Year']== yr]
# # # weather_df.dtypes

# # Merge GDP and Weather DataFrames
merge_df = pd.merge(weather_df, gdp_v2_df, how='left', on=['Year', 'ISO3'])
# # merge_df = pd.concat([weather_df, gdp2_df_output], join='outer')
merge_df.head(50)
merge_df.to_csv('Resources/Merge_data_2005-2015.csv')
merge_df

# # Delete NAs
# DeleteCountries = ['ERI', 'PRK', 'NCL', 'SOM', 'SSD', 'SYR', 'VEN']
# merge_df[~merge_df['ISO3'].isin([3, 6])]
# merge_df = merge_df.loc[weather_df['Year']== yr]
# # merge_df = merge_df.dropna(how='any')
# # merge_df
# merge_df.to_csv('Resources/Merge_data_2005-2015.csv')
# merge_df

# Generate the GDP change column
# for index, row in gdp_v2_df.iterrows():
#     gdp_v2_df['GDP_change'] = ((index['GDP']) - (index-1['GDP'])) / (index['GDP'])
# gdp_v2_df

# for i in range(len(gdp_v2_df)):
#     if gdp_v2_df['Country'] in ListofCountries:
#     gdp_v2_df['GDP_change'] = gdp_v2_df['GDP'][i] - gdp_v2_df['GDP'][i-1]/gdp_v2_df['GDP'][i]

,ISO3,Country,Year,Temp_Farenheit,Rainfall_mm,GDP
0,AFG,Afghanistan,2005,56.087945,29.609967,6.209138e+09
1,AFG,Afghanistan,2006,57.501470,27.120458,6.971286e+09
2,AFG,Afghanistan,2007,56.320054,25.329681,9.747880e+09
3,AFG,Afghanistan,2008,56.755070,21.505365,1.010923e+10
4,AFG,Afghanistan,2009,56.807912,31.106477,1.243909e+10
...,...,...,...,...,...,...
2140,ZWE,Zimbabwe,2011,71.085215,63.126104,1.410192e+10
2141,ZWE,Zimbabwe,2012,71.438135,45.312368,1.711485e+10
2142,ZWE,Zimbabwe,2013,70.923830,51.234424,1.909102e+10
2143,ZWE,Zimbabwe,2014,71.078870,50.608135,1.949552e+10


In [8]:
# # # Make Line plots
# YearGDPDict['Andorra']
# CountryTempDict['Andorra']
PlotYrList = [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
plt.plot(PlotYrList, YearGDPDict['Finland'], marker = 'o', color = 'red', linewidth =1.5, label = 'Savings')

KeyError: 'Finland'